In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import random
random.seed(420)


url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_test.data'
df_test = pd.read_csv(url, header=None, sep =' ')
df_test = df_test.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.data'
X_train = pd.read_csv(url, header=None, sep =' ')
X_train = X_train.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.labels'
y_train = pd.read_csv(url, header=None)

In [ ]:
from sklearn.preprocessing import LabelEncoder

y_train2 = LabelEncoder().fit_transform(y_train)
y_train2

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
from scipy.stats import uniform, randint


grid_params_rs = [{
    "gbc__n_estimators": randint(1,500),
    "gbc__learning_rate": uniform(0.01,0.99),
    "gbc__max_leaf_nodes":  randint(2,50),
    "gbc__min_samples_leaf": randint(1,50),
    "gbc__max_depth": randint(1,15)
}]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier


pipe = Pipeline([('anova', SelectKBest(score_func=f_classif,k=15)), ('gbc', GradientBoostingClassifier(random_state=21))])

results_rs = pd.DataFrame()
grid = RandomizedSearchCV(pipe,grid_params_rs, scoring='balanced_accuracy', random_state=37, n_iter=50)

In [ ]:
grid.fit(X_train, y_train2)

RandomizedSearchCV(estimator=Pipeline(steps=[('anova', SelectKBest(k=15)),
                                             ('gbc',
                                              GradientBoostingClassifier(random_state=21))]),
                   n_iter=50,
                   param_distributions=[{'gbc__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cdef1c0ff40>,
                                         'gbc__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cdebba90fa0>,
                                         'gbc__max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cdef1c0eb60>,
                                         'gbc__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cdebba91240>,
                                         'gbc__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cdebc1f3a30>}],
                   random_state=37, scoring='balanced_accuracy')

In [ ]:
results_rs["params"] = grid.cv_results_["params"]
results_rs["a_scores"] = grid.cv_results_["mean_test_score"]

In [ ]:
results_rs.to_csv("/drive/My Drive/AutoML/hw2/grad_boost/gb_pipe_results_rs.csv",index=False)

In [ ]:
grid.best_score_

0.858